In [1]:
import splitfolders 

#### input dataset that want to split
input_folder = 'DATASET_3'  

# output_folder= 'Splitted_Dataset_3'

splitfolders.ratio(input_folder, output= output_folder, seed=1337, ratio = (0.9, 0.0, 0.1))

Copying files: 13342 files [01:33, 142.01 files/s]


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras
from tensorflow.keras import backend as k
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import glob
import numpy as np
import json

with open('model.json', 'r') as json_file:
        json_savedModel= json_file.read()

    
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('model.h5')

In [7]:
def get_datagen():    
    datagen = ImageDataGenerator(
        #rescale=1./255,
        rotation_range=16,
        width_shift_range=0.1,
        height_shift_range=0.1,
#         horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.1
    )
    test_datagen = ImageDataGenerator(
        #rescale=1./255,
    )
    return datagen,test_datagen


def get_data(train_datagen,test_datagen):
    train_it = train_datagen.flow_from_directory('Splitted_Dataset_3/train', class_mode='categorical',target_size=(224, 224))

    test_it = test_datagen.flow_from_directory('Splitted_Dataset_3/test', class_mode='categorical',target_size=(224, 224))
    return train_it,test_it

In [8]:
def get_model(vgg): 
    
    x = tf.keras.layers.Flatten()(vgg.output)
    # three hidden layers
    x = tf.keras.layers.Dense(256, activation='selu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    # final softmax layer with two categories (dog and cat)
    predictions = tf.keras.layers.Dense(52, activation='softmax')(x)
    model = tf.keras.Model(inputs=vgg.input,outputs=predictions)
    model.summary()
    return model


In [9]:
def get_Resnet():
    from tensorflow.keras.applications import ResNet50
    vgg = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='avg')
    for i in range(0,len(vgg.layers)-20):
        vgg.layers[i].trainable = False
    vgg.summary()
    return vgg

In [10]:
datagen,test_datagen=get_datagen()
train_it,test_it=get_data(datagen,test_datagen)

Found 11984 images belonging to 52 classes.
Found 1358 images belonging to 52 classes.


In [11]:
def train(ep):
    model = get_Resnet()
    model=get_model(model)
    model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adamax(),
                      metrics=['acc'])
    history = model.fit(train_it, epochs = ep,steps_per_epoch=16, validation_data=test_it, validation_steps=8)
    return model,history

In [13]:
train(100)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

...

 conv4_block2_add (Add)         (None, 14, 14, 1024  0           ['conv4_block1_out[0][0]',       
                                )                                 'conv4_block2_3_bn[0][0]']      
                                                                                                  
 conv4_block2_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block2_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block3_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block2_out[0][0]']       
                                                                                                  
 conv4_block3_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block3_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block5_add (Add)         (None, 14, 14, 1024  0           ['conv4_block4_out[0][0]',       
                                )                                 'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_block5_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block5_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block5_out[0][0]']       
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization) 

                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 7, 7, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 7, 7, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block2_add[0][0]']       
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block2_out[0][0]']       
          

16/16 [==============================] - 37s 2s/step - loss: 0.0805 - acc: 0.9727 - val_loss: 0.0544 - val_acc: 0.9844
Epoch 89/100
16/16 [==============================] - 38s 2s/step - loss: 0.0484 - acc: 0.9844 - val_loss: 0.1112 - val_acc: 0.9609
Epoch 90/100
16/16 [==============================] - 37s 2s/step - loss: 0.0902 - acc: 0.9707 - val_loss: 0.0620 - val_acc: 0.9805
Epoch 91/100
16/16 [==============================] - 37s 2s/step - loss: 0.0800 - acc: 0.9785 - val_loss: 0.0680 - val_acc: 0.9805
Epoch 92/100
16/16 [==============================] - 37s 2s/step - loss: 0.0367 - acc: 0.9902 - val_loss: 0.0632 - val_acc: 0.9688
Epoch 93/100
16/16 [==============================] - 37s 2s/step - loss: 0.0513 - acc: 0.9824 - val_loss: 0.0524 - val_acc: 0.9883
Epoch 94/100
16/16 [==============================] - 37s 2s/step - loss: 0.0828 - acc: 0.9778 - val_loss: 0.1138 - val_acc: 0.9727
Epoch 95/100
16/16 [==============================] - 37s 2s/step - loss: 0.0866 - acc: 0

(<keras.engine.functional.Functional at 0x2648a464af0>,
 <keras.callbacks.History at 0x2648a474c40>)

In [14]:
def save_model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
save_model(model)

In [27]:
names = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '6', '7', '8', '9']

y_preds = []
y_real = []
counter=1
fig = plt.figure(figsize=(100,100))
for filename in glob.glob('data/test/1/*.jpg'):
    #im = Image.open(filename)
    img = image.load_img(filename, target_size=(224, 224))
    img_array = image.img_to_array(img)
    y_real.append(5)
    img_batch = np.expand_dims(img_array, axis=0)
    arr = model.predict(img_batch)
    maxi = 0
    cur=-1
    for i in range (0,52):
        if(arr[0][i]>maxi):
            maxi = arr[0][i]
            cur=i
    y_preds.append(cur)
    print(names[cur]+' '+str(maxi))
    if(names[cur]!='1'):
        fig.add_subplot(30,1,counter)
        counter=counter+1
        plt.imshow(img)
        plt.title(names[cur]+str(maxi))

print(counter-1)

0


<Figure size 7200x7200 with 0 Axes>

In [28]:
preds = model.predict(test_it)

43/43 [==============================] - 52s 1s/step


In [29]:
preds

array([[7.0984152e-06, 9.9622911e-01, 7.1445096e-04, ..., 1.6561197e-07,
        4.4967391e-04, 2.7642219e-04],
       [6.6942460e-09, 1.3349000e-08, 3.5418605e-08, ..., 6.4115939e-09,
        4.9733963e-08, 6.2184832e-08],
       [3.5748897e-08, 6.2122893e-08, 4.4373607e-08, ..., 1.0616774e-07,
        2.6539115e-07, 1.6848644e-07],
       ...,
       [2.6577485e-10, 7.9881822e-08, 1.3779089e-07, ..., 1.6369660e-07,
        1.5065041e-09, 1.1266641e-08],
       [5.1006849e-04, 2.5029090e-04, 6.6800055e-04, ..., 5.2419037e-04,
        4.2380032e-04, 7.1861837e-03],
       [1.6798085e-08, 4.4049531e-07, 3.3463886e-07, ..., 2.4900211e-09,
        1.0859678e-07, 1.5552915e-08]], dtype=float32)

In [17]:
test_fin_datagen = ImageDataGenerator(
)
test_it_fin = test_fin_datagen.flow_from_directory('Splitted_Dataset_3/test',shuffle=False, class_mode='categorical',target_size=(224, 224),batch_size=1)
y_preds = model.predict(test_it_fin)

Found 1358 images belonging to 52 classes.
1358/1358 [==============================] - 110s 81ms/step


In [18]:
y_real = test_it_fin.classes
y_real

array([ 0,  0,  0, ..., 51, 51, 51])

In [19]:
y_preds = np.argmax(y_preds,axis=1)

In [20]:
y_preds

array([ 0,  0,  0, ..., 51, 51, 13], dtype=int64)

NameError: name 'confusion_matrix' is not defined